In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 
from statsmodels.graphics.api import abline_plot 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn import linear_model, preprocessing 
import warnings

In [ ]:
goals = pd.read_csv('/Users/danielle.arneson/Desktop/goals.csv')

In [ ]:
goals.info()

In [ ]:
#df clean up
goals['x6_week_engagement_modality__c'] = goals['x6_week_engagement_modality__c'].str.lower()
goals['discontinued_coaching__c'] = goals['discontinued_coaching__c'].str.lower()
goals['program_name__c'] = goals['program_name__c'].str.lower()
goals['x6_week_engagement_goal_type__c'] = goals['x6_week_engagement_goal_type__c'].str.lower()
goals['overall_goal_status__c'] = goals['overall_goal_status__c'].str.lower()
goals['x6_week_engagement_modality__c'] = goals['x6_week_engagement_modality__c'].str.replace("(", "_")
goals['x6_week_engagement_modality__c'] = goals['x6_week_engagement_modality__c'].str.replace("-", "_")
goals['x6_week_engagement_modality__c'] = goals['x6_week_engagement_modality__c'].str.replace(")", "")
goals['x6_week_engagement_modality__c'] = goals['x6_week_engagement_modality__c'].str.replace(" ", "_")
goals['overall_goal_status__c'] = goals['overall_goal_status__c'].str.replace(" ", "_")
goals['program_name__c'] = goals['program_name__c'].str.replace(" ", "_")
goals['discontinued_coaching__c'] = goals['discontinued_coaching__c'].str.replace(" ", "_")
goals.head()

In [ ]:
goals['x6_week_engagement_modality__c'].unique()

In [ ]:
goals=goals.replace(to_replace =['recorded_session'],  
                            value ='group_coaching__webinar_series')
goals=goals.replace(to_replace =["one_on_one__telephonic"],  
                            value ="one_on_one")

In [ ]:
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'one_on_one__video_coaching'].index) 
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'one_on_one__1x_event'].index) 
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'biometric_event_teachable_moment__1x_event'].index) 
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'one_time_company_event'].index)
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'one_on_one_pgi'].index)
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'group_coaching__one_time_event'].index)
goals = goals.drop(goals[goals.x6_week_engagement_modality__c == 'group_coaching__onsite_series'].index)

In [ ]:
goals['x6_week_engagement_modality__c'].unique()

In [ ]:
goals = goals[goals['x6_week_engagement_modality__c'].notna()]

In [ ]:
goals['discontinued_coaching__c'] = goals['discontinued_coaching__c'].fillna(0)

In [ ]:
goals['discontinued_coaching__c'].unique()

In [ ]:
goals['goal_completed__c'] = goals['goal_completed__c'].fillna(0)

In [ ]:
goals['last_engagement_activity__c'] = goals['last_engagement_activity__c'].fillna(0)

In [ ]:
goals['createddate']=pd.to_datetime(goals['createddate'])

In [ ]:
goals = goals[goals['createddate'] > '2017-01-01 12:00:00']

In [ ]:
goals.head(50)

In [ ]:
goals['x6_week_engagement_modality__c'].describe()

In [ ]:
goals['overall_goal_status__c'].describe()

In [ ]:
sns.countplot(x="overall_goal_status__c", data=goals)

In [ ]:
status = goals.groupby(['x6_week_engagement_modality__c','overall_goal_status__c']).size().reset_index(name='counts')
print(status)

In [ ]:
status.loc['Total'] = pd.Series(status['counts'].sum(), index = ['counts'])
print (status)

In [ ]:
data = [5351,20545,19138,602452,1433,10102,402]
series = pd.Series(data, index=("Complete", "Dismissed", "In Progress", "No Action", "Partially Complete", "Reg. Phase", "Rollover"));

 

# Create a pie chart

series.plot.pie(label="", title="One on One Coaching Goals");

 

plt.show(block=True);

In [ ]:
data2 = [536398,363276,19138,602452,1433,10102,402]
series = pd.Series(data2, index=("Complete", "Dismissed", "In Progress", "No Action", "Partially Complete", "Reg. Phase", "Rollover"));

 

# Create a pie chart

series.plot.pie(label="", title="Group Coaching Goals");

 

plt.show(block=True);

In [ ]:
list(status.columns.values)

In [ ]:
Total = status['counts'].sum()

In [ ]:
status['percent'] = (status['counts'] / Total.sum() * 100)
status

In [ ]:
status= status.groupby('overall_goal_status__c')
status

In [ ]:
# Plot the data with Matplotlib defaults
plt.plot(x, y)
plt.legend('ABCDEF', ncol=2, loc='upper left');

In [ ]:
status.plot.bar()

In [ ]:
modality =goals.groupby(['x6_week_engagement_modality__c','overall_goal_status__c']).count()
modality.head(50)

In [ ]:
complete3 = pd.get_dummies(status['x6_week_engagement_goal_type__c'])

In [ ]:
complete = goals[['x6_week_engagement_modality__c','overall_goal_status__c']].copy()
complete

In [ ]:
complete = complete.groupby('overall_goal_status__c').count()
complete

In [ ]:
complete.reset_index(inplace=True)
complete.head()

In [ ]:
complete.columns

In [ ]:
complete2 = pd.get_dummies(complete['overall_goal_status__c'])

In [ ]:
complete2.columns

In [ ]:
complete.head()

In [ ]:
complete = pd.concat([complete, complete2], axis=1)
complete.head()

In [ ]:
X= complete[['complete']]
y= complete[['x6_week_engagement_modality__c']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.67, random_state= 123)
print(X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

In [ ]:
model = linear_model.LinearRegression(normalize=True)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print(model.score(X_train, y_train))

In [ ]:
sns.countplot(x="overall_goal_status__c", data=complete)

In [ ]:
data3 = [724448,417766,20344,607507,2077]
series = pd.Series(data3, index=("Complete", "Dismissed", "In Progress", "No Action", "Partially Complete"));

 

# Create a pie chart

series.plot.pie(label="", title="Total overall goal status");

 

plt.show(block=True);

In [ ]:
goals[['last_engagement_activity__c', 'createddate']].describe()

In [ ]:
goals['last_engagement_activity__c'] = pd.to_datetime(goals['last_engagement_activity__c'])

In [ ]:
activity = goals[(goals.overall_goal_status__c != 'no_action') & (goals.overall_goal_status__c != 'registration_phase')]
activity.head()

In [ ]:
activity = activity['completion_time'] = activity['last_engagement_activity__c'] - activity['createddate']

In [ ]:
activity.head()

In [ ]:
activity["Time"] = pd.to_datetime(activity['Time'])
activity.plot(x="Time", y=["completion_time"])
plt.show()